<a href="https://colab.research.google.com/github/claudiourbani/esercizi-di-programmazione-javascript/blob/master/BioMarkjet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os

DATA_FILE = "biomarket_data.json"


def load_data():
    """Load product and sales data from file."""
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r") as f:
            return json.load(f)
    return {"products": [], "sales": []}


def save_data(data):
    """Save current data to file."""
    with open(DATA_FILE, "w") as f:
        json.dump(data, f)


def find_product(products, name):
    """Find a product by name."""
    for product in products:
        if product["name"].lower() == name.lower():
            return product
    return None


def add_product(data):
    """Add or update a product in stock."""
    name = input("Nome del prodotto: ").strip()
    try:
        quantity = int(input("Quantità: "))
        if quantity <= 0:
            raise ValueError
    except ValueError:
        print("Quantità non valida.")
        return

    existing = find_product(data["products"], name)
    if existing:
        existing["quantity"] += quantity
    else:
        try:
            purchase_price = float(input("Prezzo di acquisto: "))
            sale_price = float(input("Prezzo di vendita: "))
        except ValueError:
            print("Prezzo non valido.")
            return
        data["products"].append({
            "name": name,
            "quantity": quantity,
            "purchase_price": purchase_price,
            "sale_price": sale_price
        })
    print(f"AGGIUNTO: {quantity} X {name}")


def list_products(data):
    """List all products."""
    print("PRODOTTO\tQUANTITA'\tPREZZO")
    for p in data["products"]:
        print(f"{p['name']}\t{p['quantity']}\t€{p['sale_price']}")


def register_sale(data):
    """Register a sale of one or more products."""
    sale = []
    total = 0
    while True:
        name = input("Nome del prodotto: ").strip()
        product = find_product(data["products"], name)
        if not product:
            print("Prodotto non trovato.")
            continue
        try:
            quantity = int(input("Quantità: "))
        except ValueError:
            print("Quantità non valida.")
            continue
        if quantity <= 0 or quantity > product["quantity"]:
            print("Quantità non disponibile.")
            continue
        product["quantity"] -= quantity
        line_total = round(quantity * product["sale_price"], 2)
        sale.append({
            "name": name,
            "quantity": quantity,
            "unit_price": product["sale_price"],
            "purchase_price": product["purchase_price"]
        })
        total += line_total
        more = input("Aggiungere un altro prodotto ? (si/no): ").strip().lower()
        if more != "si":
            break
    data["sales"].append(sale)
    print("VENDITA REGISTRATA")
    for item in sale:
        print(f"- {item['quantity']} X {item['name']}: €{item['unit_price']}")
    print(f"Totale: €{round(total, 2)}")


def calculate_profits(data):
    """Calculate and print gross and net profits."""
    gross = 0
    net = 0
    for sale in data["sales"]:
        for item in sale:
            gross += item["quantity"] * item["unit_price"]
            net += item["quantity"] * (item["unit_price"] - item["purchase_price"])
    print(f"Profitto: lordo=€{round(gross, 2)} netto=€{round(net, 2)}")


def print_help():
    """Print available commands."""
    print("I comandi disponibili sono i seguenti:")
    print("aggiungi: aggiungi un prodotto al magazzino")
    print("elenca: elenca i prodotto in magazzino")
    print("vendita: registra una vendita effettuata")
    print("profitti: mostra i profitti totali")
    print("aiuto: mostra i possibili comandi")
    print("chiudi: esci dal programma")


def main():
    data = load_data()
    while True:
        cmd = input("Inserisci un comando: ").strip().lower()
        if cmd == "aggiungi":
            add_product(data)
        elif cmd == "elenca":
            list_products(data)
        elif cmd == "vendita":
            register_sale(data)
        elif cmd == "profitti":
            calculate_profits(data)
        elif cmd == "aiuto":
            print_help()
        elif cmd == "chiudi":
            save_data(data)
            print("Bye bye")
            break
        else:
            print("Comando non valido")
            print_help()


if __name__ == "__main__":
    main()